# References

# Modified from KEFE's preprocess_review.py

In [1]:
!python --version
!pwd

Python 3.6.3 :: Anaconda, Inc.
/gpfs/space/home/enlik/GitRepo/master-thesis-2021/notebooks


In [2]:
from utils import *
from pprint import pprint

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

config = get_config('config.yaml')

[nltk_data] Downloading package stopwords to
[nltk_data]     /gpfs/space/home/enlik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/gpfs/space/home/enlik/.local/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Let's Get Started

## Bolt (Both Apple App Store and Google Play Store

In [3]:
df = read_csv_from_gdrive(config['csv_input']['bolt_google'])
total_reviews = len(df)
unique_users  = len(df['userName'].unique())
unknown_users = len(df[df['userName']=='A Google user'])
mean = df['score'].mean()

print(f'Total English reviews: {total_reviews} \n')
print(f'Total unique users : {unique_users}')
print(f'Total unknown users: {unknown_users}')
print(f'Total users who gave multiple reviews: {total_reviews - unique_users - unknown_users}\n')
print(f'Average rating for this app based on the textual reviews: {round(mean,2)} \n')

Total English reviews: 51907 

Total unique users : 46872
Total unknown users: 4780
Total users who gave multiple reviews: 255

Average rating for this app based on the textual reviews: 3.96 



In [4]:
df2 = read_csv_from_gdrive(config['csv_input']['bolt_apple'])
total_reviews = len(df2)
unique_users  = len(df2['userName'].unique())
mean = df2['rating'].mean()

print(f'Total English reviews: {total_reviews} \n')
print(f'Total unique users : {unique_users}')
print(f'Total users who gave multiple reviews: {total_reviews - unique_users}\n')
print(f'Average rating for this app based on the textual reviews: {round(mean,2)} \n')

Total English reviews: 3154 

Total unique users : 3149
Total users who gave multiple reviews: 5

Average rating for this app based on the textual reviews: 3.02 



In [5]:
# merge both apple and google
google = df.content
apple = df2.review
bolt_reviews = google.append(apple)
total_reviews = len(bolt_reviews)

print(f'Total English reviews: {total_reviews} \n')

Total English reviews: 55061 



In [6]:
cleaned_docs = remove_things(bolt_reviews)

2021-06-08 22:31:07,464 : ERROR : Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/storage/software/python-3.6.3/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-54edec6e7e09>", line 1, in <module>
    cleaned_docs = remove_things(bolt_reviews)
  File "/gpfs/space/home/enlik/GitRepo/master-thesis-2021/notebooks/utils.py", line 109, in remove_things
    return text.map(remove_digits_lower).map(remove_punc).map(remove_repeats)
  File "/gpfs/space/home/enlik/.local/lib/python3.6/site-packages/pandas/core/series.py", line 3630, in map
    new_values = super()._map_values(arg, na_action=na_action)
  File "/gpfs/space/home/enlik/.local/lib/python3.6/site-packages/pandas/core/base.py", line 1145, in _map_values
    new_values = map_f(values, mapper)
  File "pandas/_libs/lib.pyx", line 2329, in pandas._libs.lib.map_infer
  File "/gpfs/space/home/enlik/GitRepo/master-thesis-2021/notebooks/utils.py", line 106

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
lists_of_words = list(sentences_to_words(cleaned_docs))
lists_of_words_no_stops = remove_stopwords(lists_of_words)

In [ ]:
ngrams = make_bigrams(lists_of_words_no_stops)

In [ ]:
data_lemmatized = lemmatize(ngrams, allowed_postags=['NOUN'])

In [ ]:
corpora.Dictionary(data_lemmatized)

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
term_doc = [id2word.doc2bow(text) for text in texts]

# View
print(term_doc[:1])

In [ ]:
id2word[0]

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in term_doc[:1]]


In [ ]:
tf_idf = models.TfidfModel(term_doc, smartirs='ntc')[term_doc]
tf_idf[0]

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in tf_idf[:1]]

# Save pre-processed data into binary Pickle file

In [ ]:
import pickle

output_path = 'preprocessed_data/'

## data_lemmatized

In [ ]:
with open(output_path + 'data_lemmatized.pkl', 'wb') as f:
    pickle.dump(data_lemmatized, f)

In [ ]:
df = pd.read_pickle('preprocessed_data/data_lemmatized.pkl')
df

## dictionary

In [ ]:
with open(output_path + 'dictionary.pkl', 'wb') as f:
    pickle.dump(id2word, f)

## term_doc

In [ ]:
with open(output_path + 'term_doc.pkl', 'wb') as f:
    pickle.dump(term_doc, f)

## tf_idf

In [ ]:
with open(output_path + 'tf_idf.pkl', 'wb') as f:
    pickle.dump(tf_idf, f)